# 1. 기본 설정

## 0) Chrome 설치

In [0]:
%%sh
set -eux
wget -q -O - https://dl.google.com/linux/linux_signing_key.pub | apt-key add -
echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" > /etc/apt/sources.list.d/google-chrome.list
apt-get update -y
apt-get install -y google-chrome-stable
google-chrome --version

+ wget -q -O - https://dl.google.com/linux/linux_signing_key.pub
+ apt-key add -


OK


+ echo deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main
+ apt-get update -y


Get:1 http://dl.google.com/linux/chrome/deb stable InRelease [1825 B]
Hit:2 https://repos.azul.com/zulu/deb stable InRelease
Get:3 http://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1209 B]
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 3034 B in 2s (1970 B/s)
Reading package lists...


W: http://dl.google.com/linux/chrome/deb/dists/stable/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: https://repos.azul.com/zulu/deb/dists/stable/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
+ apt-get install -y google-chrome-stable


Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  fonts-liberation libgbm1 libvulkan1 libwayland-server0 libxcb-randr0
  mesa-vulkan-drivers
The following NEW packages will be installed:
  fonts-liberation google-chrome-stable libgbm1 libvulkan1 libwayland-server0
  libxcb-randr0 mesa-vulkan-drivers
0 upgraded, 7 newly installed, 0 to remove and 40 not upgraded.
Need to get 132 MB of archives.
After this operation, 450 MB of additional disk space will be used.
Get:1 http://dl.google.com/linux/chrome/deb stable/main amd64 google-chrome-stable amd64 140.0.7339.127-1 [121 MB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-liberation all 1:1.07.4-11 [822 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libwayland-server0 amd64 1.20.0-1ubuntu0.1 [34.3 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxcb-randr0 amd64 1.14-3ubuntu3 [18.3 kB]
Get:5 http://archi

debconf: delaying package configuration, since apt-utils is not installed


Fetched 132 MB in 4s (33.2 MB/s)
Selecting previously unselected package fonts-liberation.
(Reading database ... 93510 files and directories currently installed.)
Preparing to unpack .../0-fonts-liberation_1%3a1.07.4-11_all.deb ...
Unpacking fonts-liberation (1:1.07.4-11) ...
Selecting previously unselected package libwayland-server0:amd64.
Preparing to unpack .../1-libwayland-server0_1.20.0-1ubuntu0.1_amd64.deb ...
Unpacking libwayland-server0:amd64 (1.20.0-1ubuntu0.1) ...
Selecting previously unselected package libxcb-randr0:amd64.
Preparing to unpack .../2-libxcb-randr0_1.14-3ubuntu3_amd64.deb ...
Unpacking libxcb-randr0:amd64 (1.14-3ubuntu3) ...
Selecting previously unselected package libgbm1:amd64.
Preparing to unpack .../3-libgbm1_23.2.1-1ubuntu3.1~22.04.3_amd64.deb ...
Unpacking libgbm1:amd64 (23.2.1-1ubuntu3.1~22.04.3) ...
Selecting previously unselected package libvulkan1:amd64.
Preparing to unpack .../4-libvulkan1_1.3.204.1-2_amd64.deb ...
Unpacking libvulkan1:amd64 (1.3.204.

+ google-chrome --version


Google Chrome 140.0.7339.127 


## 1) 파이썬 패키지 설치 및 로드

In [0]:
%pip install --quiet selenium==4.21.0 webdriver-manager==4.0.2 beautifulsoup4==4.12.3 lxml==5.2.2 pandas==2.2.2

dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
import os, re, json, time, random, pandas as pd
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from datetime import datetime, timezone, timedelta
from pyspark.sql.functions import col, to_date
from pyspark.sql import functions as F
from pyspark.sql import types as T
import pytz

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# 2. 크롤링 + 적재

## 0) 공통설정

### 공통 설정/함수(드라이버 + 무한스크롤 + 상세파서)

In [0]:
BASE = "https://jumpit.saramin.co.kr"
HEADLESS = True
DETAIL_DELAY = (0.7, 1.8)  # 요청 예절용 랜덤 대기
SEOUL = pytz.timezone("Asia/Seoul")

# 크롬 드라이버 설정
def start_driver():
    opts = Options()
    if HEADLESS:
        opts.add_argument("--headless=new")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--window-size=1920,1080")
    opts.add_argument("--lang=ko-KR")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=opts)
    driver.set_page_load_timeout(60)
    return driver

# 무한 스크롤로 모든 공고 링크 수집
def scroll_and_collect_links_all(driver, start_url, idle_rounds=8, max_minutes=30, settle_sleep=1.2):
    driver.get(start_url)
    WebDriverWait(driver, 25).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.sc-d609d44f-0.grDLmW a[href^="/position/"]'))
    )
    seen = set(); no_change = 0; t0 = time.monotonic() # 공고 url 수집용 집합 변수
    while True:
        anchors = driver.find_elements(By.CSS_SELECTOR, 'div.sc-d609d44f-0.grDLmW a[href^="/position/"]') # url 저장
        before = len(seen)
        for a in anchors:
            href = a.get_attribute("href")
            if href and href.startswith("/position/"):
                href = urljoin(BASE, href)
            if href and "/position/" in href:
                seen.add(href)

        last_h = driver.execute_script("return document.body.scrollHeight") # 스크롤 실행 전 스크롤 길이 저장
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 스크롤 실행
        time.sleep(settle_sleep)
        new_h = driver.execute_script("return document.body.scrollHeight") # 스크롤 실행 후 스크롤 길이 저장

        grew_in_height = new_h > last_h
        grew_in_links = len(seen) > before
        no_change = no_change + 1 if (not grew_in_height and not grew_in_links) else 0 # 스크롤 길이 변경 여부 확인

        if no_change >= idle_rounds: # 스크롤 종료 여부 확인
            print(f"[done] 끝 도달 – {len(seen)}개 링크")
            break
        if (time.monotonic() - t0)/60 > max_minutes: # 시간 종료 여부 확인
            print(f"[warn] 시간 상한 {max_minutes}분 – {len(seen)}개 링크까지만 수집")
            break
    return list(seen) # url 리턴

# 유틸: 텍스트 정리
def clean_text(s):
    if s is None: return ""
    # 공백/줄바꿈 정리
    return re.sub(r'\n{3,}', '\n\n', re.sub(r'[ \t]+', ' ', s)).strip()

def text_from_dd(dd):
    # dd 내부가 <pre> 또는 리스트/텍스트 혼합일 수 있음
    if not dd: return ""
    # 링크의 href 텍스트도 함께 붙이기
    for a in dd.select('a[href]'):
        if a.get_text(strip=True) and a['href']:
            a.string = f"{a.get_text(strip=True)} ({a['href']})"
    txt = dd.get_text("\n", strip=True)
    return clean_text(txt)

# ---- 상세 페이지 파서 (루트: .sc-3ef60426-5.bBiQPb) ----
def parse_detail_html(html, url):
    soup = BeautifulSoup(html, "lxml")
    root = soup.select_one('.sc-3ef60426-5.bBiQPb')
    if not root:
        # 백업: 접두/접미 해시가 변할 가능성 대비
        root = soup.select_one('[class*="sc-3ef60426-"][class*="bBiQPb"], section.sc-3ef60426-5')
    if not root:
        root = soup

    # 공고명 / 기업명 / 축하금
    title = root.select_one('h1')
    company_anchor = root.select_one('a.name')
    congrats = root.select_one('[title*="취업축하금"], [class*="취업축하금"], span:contains("취업축하금")')
    공고명 = title.get_text(strip=True) if title else ""
    기업명 = company_anchor.get_text(strip=True) if company_anchor else ""
    company_href = ""
    if company_anchor and company_anchor.get("href"):
        h = company_anchor["href"]
        company_href = h if h.startswith("http") else urljoin(BASE, h)

    점핏취업축하금 = ""
    if congrats:
        점핏취업축하금 = clean_text(congrats.get_text(strip=True))

    # 주요 상세 dl (기술스택/주요업무/자격요건/우대사항/복지 및 혜택/채용절차 및 기타 지원 유의사항)
    label_map = {
        "기술스택":"기술스택",
        "주요업무":"주요업무",
        "자격요건":"자격요건",
        "우대사항":"우대사항",
        "복지 및 혜택":"복지 및 혜택",
        "채용절차 및 기타 지원 유의사항":"채용절차 및 기타 지원 유의사항",
    }
    values = {v:"" for v in label_map.values()}
    
    for dl in root.select('div.position_info dl'):
        dt = dl.select_one('dt')
        dd = dl.select_one('dd')
        if not dt or not dd: 
            continue
        key = dt.get_text(strip=True)
        if key in label_map:
            # 기술스택은 아이콘 div나 텍스트가 섞임 → 보강
            if key == "기술스택":
                chips = [clean_text(x.get_text(" ", strip=True)) for x in dd.select('[class*="sc-d9de2de1-0"], [class*="stack"], img[alt]')]
                chips = [c for c in chips if c]
                if chips:
                    values[label_map[key]] = ", ".join(dict.fromkeys(chips))  # 중복 제거 (파이썬 3.7부터 딕셔너리는 삽입 순서를 보장함. 따라서 dict.fromkeys()를 사용하면 중복을 제거하면서 원래 리스트의 요소 순서를 그대로 유지할 수 있음)
                else:
                    values[label_map[key]] = text_from_dd(dd)
            else:
                values[label_map[key]] = text_from_dd(dd)

    # 경력/학력/마감일/근무지역
    경력 = 학력 = 마감일 = 근무지역 = ""
    for dl in root.select('div.sc-b12ae455-0 dl, dl.sc-b12ae455-1'):
        dt = dl.select_one('dt')
        dd = dl.select_one('dd')
        if not dt or not dd: continue
        k = dt.get_text(strip=True)
        if k == "경력":   경력 = text_from_dd(dd)
        elif k == "학력": 학력 = text_from_dd(dd)
        elif k == "마감일": 마감일 = text_from_dd(dd)
        elif k == "근무지역":
            # li 여러 개일 수 있음
            locs = [clean_text(li.get_text(" ", strip=True)) for li in dd.select('li')]
            근무지역 = " | ".join(locs) if locs else text_from_dd(dd)
        
    collected_at = datetime.now(SEOUL).strftime("%Y-%m-%d %H:%M:%S")

    return {
        "posting_title": 공고명,
        "company_name": 기업명,
        "hiring_bonus": 점핏취업축하금,
        "posting_tech_stack": values["기술스택"],
        "responsibilities": values["주요업무"],
        "minimum_qualifications": values["자격요건"],
        "preferred_qualifications": values["우대사항"],
        "posting_benefits": values["복지 및 혜택"],
        "hiring_process": values["채용절차 및 기타 지원 유의사항"],
        "experience": 경력,
        "education": 학력,
        "end_date": 마감일,
        "work_location": 근무지역,
        "posting_url": url,
        "posting_id": (re.search(r"/position/(\d+)", url).group(1) if re.search(r"/position/(\d+)", url) else None),
        "company_url_jumpit": company_href,
        "jumpit_posting_collection_time": collected_at
    }

# ---- 상세 페이지 크롤링 ----
def crawl_detail_pages(urls, delay_range=DETAIL_DELAY):
    rows = []
    driver = start_driver() # 하나의 웹 브라우저 인스턴스를 시작
    for i, url in enumerate(urls, 1):
        try:
            driver.get(url) #현재 URL로 이동하여 페이지를 로드
            # 루트 블록 대기
            WebDriverWait(driver, 25).until(
                EC.any_of(
                    EC.presence_of_element_located((By.CSS_SELECTOR, '.sc-3ef60426-5.bBiQPb')),
                    EC.presence_of_element_located((By.CSS_SELECTOR, '[class*="sc-3ef60426-"][class*="bBiQPb"]'))
                )
            )
            html = driver.page_source #현재 페이지 HTML 소스 코드 전체 가져옴
            rows.append(parse_detail_html(html, url))
        except TimeoutException:
            rows.append({
                "posting_title": "", "company_name": "", "hiring_bonus": "", "posting_tech_stack": "",
                "responsibilities":"", "minimum_qualifications":"", "preferred_qualifications":"", "posting_benefits":"",
                "hiring_process":"", "experience":"", "education":"",
                "end_date":"", "work_location":"", "posting_url": url, "posting_id": None, "company_url_jumpit": None,
                "jumpit_posting_collection_time": None
            })  
        finally:
            time.sleep(random.uniform(*delay_range))
    driver.quit()
    return rows


In [0]:
# ---- DB 저장 ----
def insert_data(
    data: pd.DataFrame,
    schema: str,
    table: str,
    mode: str = "overwrite",
    # --- 연결 대상 선택 ---
    target: str = "cloud",   # "cloud" 또는 "vm"
    # --- 공통 설정 ---
    port: str = "5432",
    # --- 클라우드(DBaaS) 설정 ---
    cloud_host: str = "cloud-postgredb-server.postgres.database.azure.com",
    cloud_db: str = "postgres",
    cloud_user: str = "Drawbridge",         
    cloud_pass: str = "asdASD123!@#",
    # --- VM 설정 ---
    vm_host: str = "20.196.145.211",
    vm_db: str = "postgres",
    vm_user: str = "postgres",
    vm_pass: str = "asdASD123!@#",
):
    # pandas → spark DataFrame
    sdf = spark.createDataFrame(data)

    if "posting_id" in sdf.columns:
        sdf = sdf.withColumn("posting_id", col("posting_id").cast("bigint"))
    if "posting_id_collection_time" in sdf.columns:
        sdf = sdf.withColumn(
            "posting_id_collection_time",
            to_date(col("posting_id_collection_time"), "yyyy-MM-dd")
        )

    if target == "cloud":
        host = cloud_host
        db   = cloud_db
        user = cloud_user
        pwd  = cloud_pass
        # Azure DB는 SSL 필수 → sslmode=require
        jdbc_url = f"jdbc:postgresql://{host}:{port}/{db}"
        options = {
            "url": jdbc_url,
            "user": user,
            "password": pwd,
            "dbtable": f"{schema}.{table}",
            "driver": "org.postgresql.Driver",
            "sslmode": "require",
            # 성능 옵션(선택): 대량 삽입 최적화
            "reWriteBatchedInserts": "true",
            "batchsize": "5000",
        }
    elif target == "vm":
        host = vm_host
        db   = vm_db
        user = vm_user
        pwd  = vm_pass
        jdbc_url = f"jdbc:postgresql://{host}:{port}/{db}"
        options = {
            "url": jdbc_url,
            "user": user,
            "password": pwd,
            "dbtable": f"{schema}.{table}",
            "driver": "org.postgresql.Driver",
        }
    else:
        raise ValueError("target must be 'cloud' or 'vm'")

    # 쓰기
    sdf.write.format("jdbc").mode(mode).options(**options).save()

## 1) 직무 카테고리 정의

### 직무 카테고리 정의(1~21) + url 빌더

In [0]:
# 직무 카테고리 매핑
JOB_CATEGORIES = {
    1: "서버/백엔드 개발자",
    2: "프론트엔드 개발자",
    3: "웹 풀스택 개발자",
    4: "안드로이드 개발자",
    5: "게임 클라이언트 개발자",
    6: "게임 서버 개발자",
    7: "DBA",
    8: "인공지능/머신러닝",
    9: "DevOps/시스템 엔지니어",
    10: "정보보안 담당자",
    11: "QA 엔지니어",
    12: "개발 PM",
    13: "HW/임베디드",
    15: "SW/솔루션",
    16: "iOS 개발자",
    17: "웹퍼블리셔",
    18: "크로스플랫폼 앱개발자",
    19: "빅데이터 엔지니어",
    20: "VR/AR/3D",
    21: "기술지원",
    22: "블록체인"
}

def build_list_url(job_cat: int) -> str:
    # 요청한 형식: ?jobCategory={id}&sort=reg_dt
    # (복수 카테고리를 동시에 묶지 않고, "직무별로 별도 크롤"을 위해 단일 카테고리만 구성)
    return f"{BASE}/positions?jobCategory={job_cat}&sort=reg_dt"

## 2) 채용공고 링크 크롤링

### 카테고리별 목록 크롤 -> 체크포인트(JSON) + 목록 페이지 직무 카테고리 Delta 누적 기록

In [0]:
# 체크포인트 저장 디렉터리
# 체크포인트 디렉터리를 /dbfs/FileStore/jumpit_ckpt/by_category로 설정함. 이 경로는 Databricks 파일 시스템에 데이터를 저장하여 클러스터가 재시작되어도 유지됨.
ckpt_dir = "/dbfs/FileStore/jumpit_ckpt/by_category"
os.makedirs(ckpt_dir, exist_ok=True)

# 실행할 카테고리 (원하면 부분만 선택해서 실행 가능/예: [1,2,3] 등)
CATS_TO_RUN = list(JOB_CATEGORIES.keys())

rows = []  # 테이블 적재용 (직무카테고리, 링크 수집 시각)
driver = start_driver()
try:
    for cid in CATS_TO_RUN:
        list_url = build_list_url(cid)
        print(f"[{cid}] {JOB_CATEGORIES.get(cid,'')} 목록 수집 시작 → {list_url}")
        links = scroll_and_collect_links_all(
            driver, list_url,
            idle_rounds=8,    # 페이지 끝에 도달했음을 감지하는 횟수를 8회로 설정
            max_minutes=30,   #  크롤링 작업의 최대 실행 시간을 30분으로 설정
            settle_sleep=1.2  # 스크롤 후 콘텐츠가 로드될 때까지 1.2초 동안 대기
        )
        # 카테고리별 링크 체크포인트 저장
        ckpt_path = f"{ckpt_dir}/links_jobCategory_{cid}_{datetime.now(SEOUL).strftime('%Y%m%d')}.json"
        with open(ckpt_path, "w") as f:
            json.dump(links, f, ensure_ascii=False) #ensure_ascii=True (기본값)로 저장하면 한글이 유니코드 이스케이프 코드로 변환됨
        print(f"[{cid}] 링크 {len(links)}건 저장 → {ckpt_path}")

        # Databricks 런타임 타임존 기준 (SQL current_timestamp()와 동일하게 Spark 쪽에서 찍어도 됨)
        rows.append((int(cid), datetime.now(SEOUL).strftime("%Y-%m-%d %H:%M:%S")))
finally:
    driver.quit() # 드라이버 종료(메모리 누수 방지)

cols = ["job_category", "jumpit_link_collection_time"]
df = pd.DataFrame(rows, columns=cols)

for c in cols:
    if c not in df.columns:
        df[c] = None
df = df[cols]

csv_path = f"/dbfs/FileStore/jumpit_job_category_{datetime.now(SEOUL).strftime('%Y%m%d')}.csv"
df.to_csv(csv_path, index=False)
print(f"직무 카테고리별 링크 수집 시각 CSV 저장: {csv_path}")

[1] 서버/백엔드 개발자 목록 수집 시작 → https://jumpit.saramin.co.kr/positions?jobCategory=1&sort=reg_dt
[done] 끝 도달 – 254개 링크
[1] 링크 254건 저장 → /dbfs/FileStore/jumpit_ckpt/by_category/links_jobCategory_1_20250916.json
[2] 프론트엔드 개발자 목록 수집 시작 → https://jumpit.saramin.co.kr/positions?jobCategory=2&sort=reg_dt
[done] 끝 도달 – 147개 링크
[2] 링크 147건 저장 → /dbfs/FileStore/jumpit_ckpt/by_category/links_jobCategory_2_20250916.json
[3] 웹 풀스택 개발자 목록 수집 시작 → https://jumpit.saramin.co.kr/positions?jobCategory=3&sort=reg_dt
[done] 끝 도달 – 51개 링크
[3] 링크 51건 저장 → /dbfs/FileStore/jumpit_ckpt/by_category/links_jobCategory_3_20250916.json
[4] 안드로이드 개발자 목록 수집 시작 → https://jumpit.saramin.co.kr/positions?jobCategory=4&sort=reg_dt
[done] 끝 도달 – 30개 링크
[4] 링크 30건 저장 → /dbfs/FileStore/jumpit_ckpt/by_category/links_jobCategory_4_20250916.json
[5] 게임 클라이언트 개발자 목록 수집 시작 → https://jumpit.saramin.co.kr/positions?jobCategory=5&sort=reg_dt
[done] 끝 도달 – 5개 링크
[5] 링크 5건 저장 → /dbfs/FileStore/jumpit_ckpt/by_category/links_jobCategory_5_202

In [0]:
# 카테고리별 링크 저장용 딕셔너리 변수 추가
added_data = {1: None,
              2: None,
              3: None,
              4: None,
              5: None,
              6: None,
              7: None,
              8: None,
              9: None,
              10: None,
              11: None,
              12: None,
              13: None,
              15: None,
              16: None,
              17: None,
              18: None,
              19: None,
              20: None,
              21: None,
              22: None
              }

for cid in CATS_TO_RUN:
    # 데이터 읽어오기
    yesterday = datetime.now(SEOUL) - timedelta(days=1)
    date_str = yesterday.strftime("%Y%m%d")

    with open(f'/dbfs/FileStore/jumpit_ckpt/by_category/links_jobCategory_updated_{cid}_{date_str}.json', 'r') as f:
        old_file = f.read()
    with open(f"/dbfs/FileStore/jumpit_ckpt/by_category/links_jobCategory_{cid}_{datetime.now(SEOUL).strftime('%Y%m%d')}.json", 'r') as f:
        new_file = f.read()

    # 문자열(str) → 리스트 변환
    old_list = json.loads(old_file)
    new_list = json.loads(new_file)

    # old_file에 있는 링크 제거
    result_list = list(set(new_list) - set(old_list))
    added_data[cid] = result_list

    # old_list에 추가
    updated_list = old_list + result_list

    # 중복 제거 (혹시 몰라서 set 사용, 순서 보존을 위해 dict.fromkeys 활용)
    updated_list = list(dict.fromkeys(updated_list))

    print("추가된 링크 수:", len(result_list))
    print("기존 + 추가 링크 수:", len(updated_list))

    # 업데이트된 링크 csv 저장
    # 일단 원본 안건드리고 다른 이름으로 저장함
    ckpt_path = f"{ckpt_dir}/links_jobCategory_updated_{cid}_{datetime.now(SEOUL).strftime('%Y%m%d')}.json"
    with open(ckpt_path, "w") as f:
        json.dump(updated_list, f, ensure_ascii=False) #ensure_ascii=True (기본값)로 저장하면 한글이 유니코드 이스케이프 코드로 변환됨
    print(f"[{cid}] 링크 {len(updated_list)}건 저장 → {ckpt_path}")

추가된 링크 수: 64
기존 + 추가 링크 수: 313
[1] 링크 313건 저장 → /dbfs/FileStore/jumpit_ckpt/by_category/links_jobCategory_updated_1_20250916.json
추가된 링크 수: 44
기존 + 추가 링크 수: 186
[2] 링크 186건 저장 → /dbfs/FileStore/jumpit_ckpt/by_category/links_jobCategory_updated_2_20250916.json
추가된 링크 수: 9
기존 + 추가 링크 수: 61
[3] 링크 61건 저장 → /dbfs/FileStore/jumpit_ckpt/by_category/links_jobCategory_updated_3_20250916.json
추가된 링크 수: 8
기존 + 추가 링크 수: 39
[4] 링크 39건 저장 → /dbfs/FileStore/jumpit_ckpt/by_category/links_jobCategory_updated_4_20250916.json
추가된 링크 수: 0
기존 + 추가 링크 수: 6
[5] 링크 6건 저장 → /dbfs/FileStore/jumpit_ckpt/by_category/links_jobCategory_updated_5_20250916.json
추가된 링크 수: 0
기존 + 추가 링크 수: 3
[6] 링크 3건 저장 → /dbfs/FileStore/jumpit_ckpt/by_category/links_jobCategory_updated_6_20250916.json
추가된 링크 수: 6
기존 + 추가 링크 수: 30
[7] 링크 30건 저장 → /dbfs/FileStore/jumpit_ckpt/by_category/links_jobCategory_updated_7_20250916.json
추가된 링크 수: 51
기존 + 추가 링크 수: 232
[8] 링크 232건 저장 → /dbfs/FileStore/jumpit_ckpt/by_category/links_jobCategory_upd

## 추가된 공고 리스트 상세 크롤 돌리기

In [0]:
# COMMAND ----------
def run_detail_for_category(cid: int, batch_size=120):
    print(f"[{cid}]번 직무 상세 크롤 시작: {len(added_data[cid])}건")

    all_rows = []

    for s in range(0, len(added_data[cid]), batch_size):        
        part = added_data[cid][s:s+batch_size]
        rows = crawl_detail_pages(part)  # 기존 상세 크롤 함수 재사용
        all_rows.extend(rows) # 크롤링으로 얻은 데이터(rows)를 all_rows 리스트에 추가
        df_part = pd.DataFrame(rows)
        part_csv = f"/dbfs/FileStore/jumpit_posting_job{cid}_part_{s//batch_size:03d}_{datetime.now(SEOUL).strftime('%Y%m%d')}.csv"
        df_part.to_csv(part_csv, index=False)
        print(f"[{cid}] Saved part {s//batch_size:03d} → {part_csv}")

    df_all = pd.DataFrame(all_rows)
    # 기존 ‘채용공고 상세 페이지’ 스키마에 맞춰서 컬럼 순서 정렬
    cols = [
        "posting_title","company_name","hiring_bonus","posting_tech_stack","responsibilities","minimum_qualifications","preferred_qualifications",
        "posting_benefits","hiring_process","experience","education","end_date","work_location",
        "posting_url","posting_id","company_url_jumpit", "jumpit_posting_collection_time"
    ]
    # 없는 컬럼은 값 생성
    # 이 방법이 reindex()보다 속도가 더 빠르다고 함
    for c in cols:
        if c not in df_all.columns:
            df_all[c] = None # 빠지는 컬럼은 비워서라도 만들고
    df_all = df_all[cols] # 순서는 이대로
    
    # CSV 저장
    final_csv = f"/dbfs/FileStore/jumpit_posting_job{cid}_all_{datetime.now(SEOUL).strftime('%Y%m%d')}.csv"
    df_all.to_csv(final_csv, index=False)
    print(f"[{cid}] 상세 CSV 저장: {final_csv}")

    return df_all

In [0]:
df_list = []

for i in JOB_CATEGORIES.keys():
    temp_df = run_detail_for_category(i)
    temp_df['job_category'] = i
    df_list.append(temp_df)

df_all = pd.concat(df_list, ignore_index=True)

display(df_all)

[1]번 직무 상세 크롤 시작: 64건
[1] Saved part 000 → /dbfs/FileStore/jumpit_posting_job1_part_000_20250916.csv
[1] 상세 CSV 저장: /dbfs/FileStore/jumpit_posting_job1_all_20250916.csv
[2]번 직무 상세 크롤 시작: 44건
[2] Saved part 000 → /dbfs/FileStore/jumpit_posting_job2_part_000_20250916.csv
[2] 상세 CSV 저장: /dbfs/FileStore/jumpit_posting_job2_all_20250916.csv
[3]번 직무 상세 크롤 시작: 9건
[3] Saved part 000 → /dbfs/FileStore/jumpit_posting_job3_part_000_20250916.csv
[3] 상세 CSV 저장: /dbfs/FileStore/jumpit_posting_job3_all_20250916.csv
[4]번 직무 상세 크롤 시작: 8건
[4] Saved part 000 → /dbfs/FileStore/jumpit_posting_job4_part_000_20250916.csv
[4] 상세 CSV 저장: /dbfs/FileStore/jumpit_posting_job4_all_20250916.csv
[5]번 직무 상세 크롤 시작: 0건
[5] 상세 CSV 저장: /dbfs/FileStore/jumpit_posting_job5_all_20250916.csv
[6]번 직무 상세 크롤 시작: 0건
[6] 상세 CSV 저장: /dbfs/FileStore/jumpit_posting_job6_all_20250916.csv
[7]번 직무 상세 크롤 시작: 6건
[7] Saved part 000 → /dbfs/FileStore/jumpit_posting_job7_part_000_20250916.csv
[7] 상세 CSV 저장: /dbfs/FileStore/jumpit_posting_jo

posting_title,company_name,hiring_bonus,posting_tech_stack,responsibilities,minimum_qualifications,preferred_qualifications,posting_benefits,hiring_process,experience,education,end_date,work_location,posting_url,posting_id,company_url_jumpit,jumpit_posting_collection_time,job_category
웹서버 개발자 신입 채용,에프엘이에스,💰 취업축하금 50만원,"Java, React, Spring, C++",• 웹서버 개발,• 대졸 이상 • C++ 개발 경험자 • 스프링 자바 개발 경력자,"• 컴퓨터 공학 등 관련 학과 우대 • NVR, DVR, VMS 관련 개발 경험자 우대 • GoLang 등 다양한 개발 언어 사용 가능자 • javascript, 기타 Front 웹 프레임워크(React, Vue)등 지식을 가진자","• 자율 출퇴근 • 스톡옵션 지급 가능 • 맥북 지급 • 도서, 교육, 세미나 • 네스프레소 커피 머신, 맥주 무제한, 간식 무제한",-,신입,대학교졸업(4년) 이상,2025-10-13,"대전 유성구 장대로106, 제2층 제45에이호 지도보기 · 주소복사",https://jumpit.saramin.co.kr/position/51723619,51723619,https://jumpit.saramin.co.kr/company/MTc0ODEwMTU1Ng==?company_nm=에프엘이에스,2025-09-16 10:13:38,1
Back-End Engineer,(주)퓨쳐스콜레,💰 취업축하금 50만원,"JavaScript, Java, Spring Framework, MariaDB, AWS, REST API","퓨쳐스콜레 프로덕트팀에서 LiveKlass 서비스의 성장과 기술적인 혁신을 함께합니다. • LiveKlass 서비스 백엔드 시스템을 설계하고 개발합니다. • 프론트엔드 팀과 협업하여 확장 가능하고 성능이 뛰어난 시스템을 구축합니다. • AWS 클라우드 환경에서 서비스 개발, 모니터링, 운영을 수행하며 DevOps, SRE 문화를 함께 만들어갑니다. • Jira, Slack, Notion, Github 등의 협업 툴을 활용하여 비즈니스 팀과 지속적으로 협업합니다. • 새로운 기술과 방향성을 학습하고 문제 해결을 위해 함께 고민하고 협력합니다.","다음 기술과 경험을 보유하신 분을 찾습니다. • 5년 이상의 Back-end 개발 경력 또는 그에 준하는 경험 보유 ✅ Java & Spring Framework • Java 8 이상 개발 경험 • Spring Boot 기반의 웹 애플리케이션 개발 경험 • Spring MVC, Spring Data JPA, Spring Security 등의 활용 능력 ✅ 데이터베이스 (MariaDB) • MariaDB(MySQL) 설계 및 최적화 경험 • ORM(JPA, Hibernate) 활용 및 Native Query 튜닝 경험 • 데이터 마이그레이션 및 백업/복구 경험 ✅ 클라우드 & DevOps (AWS, Docker) • AWS EC2, S3, RDS(MariaDB), Lambda, SQS, SNS 등의 서비스 활용 경험 • Docker 컨테이너 및 Docker Compose 활용 경험 • ECS, EKS, K8S 등 운영 및 구축 경험 • CI/CD 파이프라인 구축 경험 (GitHub Actions, Jenkins 등) ✅ 기타 필수 기술 • RESTful API 설계 및 개발 경험 • 대규모 트래픽을 고려한 시스템 설계 경험 • Git을 활용한 협업 경험 • 단위 테스트 및 통합 테스트 작성 경험 (JUnit, Mockito 등) • 프론트엔드 개발자, 기획자 등 타 직군과 협업 경험","다음 경험이 있으시면 더욱 좋습니다. 🏗️ 아키텍처 & 성능 최적화 • 마이크로서비스 아키텍처(MSA) 경험 • 이벤트 기반 아키텍처(EDA) 경험 • 대용량 트래픽 처리 및 성능 최적화 경험 • Top & Slow Query 최적화 경험 • Redis, Kafka, RabbitMQ 등 메시징 시스템 활용 경험 🔒운영 & 보안 • Kubernetes를 활용한 서비스 운영 경험 • Spring Security 기반의 인증 및 인가 구현 경험 • Spring Vulnerability 개선 경험 • 서버 및 API 보안(OWASP Top 10) 고려한 개발 경험 🚀 기타 우대 사항 • TDD(Test-Driven Development) 경험 • 대규모 프로젝트 또는 스타트업 경험 • 코드 리뷰 및 개발 문화 개선에 대한 관심 • 자체 서비스 개발 및 운영 경험","• 성과에 대해서도 적극적으로 보상해요! 성과급, 스톡옵션 지급 • 앞으로 오래 함께 일해요! 장기근속자(매 3년) 포상 • 모든 동료들과 즐거운 시간을 보내요! 매주 금요일 성수동 맛집 탐방 • 전사가 투명하게 소통해요! 달고나 타운홀 미팅(매월 2회) • 열심히 일했으니 재미있게 놀아요! 반기에 한 번 놀샵! • 한계를 두지 말고 성장해요! 도서비 100% 지원, 세미나 100% 지원, 사내 강의 교육 지원 • 서로 존중하며 솔직하게 피드백해요! ‘영어 닉네임’으로 소통합니다. ﻿","• 채용 형태 : 정규직 (수습기간 3개월) • 급여 조건 : 연봉 (경력에 따라 협의) • 근무 시간 : 8시 ~ 10시 사이 자유롭게 출근합니다. • 제출 서류 : 이력서 및 포트폴리오 • 채용 절차 : 서류 전형 - 폰 스크리닝 (HR) - 1차 직무역량 인터뷰(대면, 실무자) - 2차 임원 인터뷰 (대면, CEO) - 처우 협의 - 합류 * 위 채용 절차는 기본 프로세스로 경우에 따라 1차 & 2차 인터뷰 One day 진행 등으로 일부 변경이 있을 수 있습니다. * 서류에 기재한 내용과 사실이 다를 경우 합격이 취소될 수 있습니다.",경력 5~20년,무관,2025-10-10,"서울 성동구 왕십리로115, 7층 지707 지도보기 · 주소복사",https://jumpit.saramin.co.kr/position/51807381,51807381,https://jumpit.saramin.co.kr/company/MzU2ODgwMTE3Ng==?company_nm=(주)퓨쳐스콜레,2025-09-16 10:13:40,1
백엔드 개발자(경력8~10년),아이도트,💰 취업축하금 50만원,"Spring, RDB, REST API, Git, AWS, Java",-자사 운영 중인 의료 인공지능 웹 서버 유지.보수 및 고도화,"-전문 학사 이상 학위를 보유하신 분 -Java, JS 등 사용 경험이 있으신분 -Spring, RDBMS등 사용 경험이 있으신 분 -RESTful API 설계 및 구현 경험이 있으신 분 -문제 해경 능력 및 분석 능력이 있으신 분","-Git, DevOps, CI/CD 경험이 있으신 분 -클라우드(AWS등) 기반 시스템 개발 경험이 있으신 분 -프론트엔드 개발 경험이 있으신 분 -테스트 코드를 적극적으로 작성하시는 분","• 급여제도 : 퇴직연금, 스톡옵션, 퇴직금, 4대 보험 • 근무 환경 : 회의실, 공기청정기, 카페테리아, 사원증, 사무용품 지급 • 지원금/보험 : 각종 경조사 지원, 헬스비 지급 • 리프레시 : 연차, 경조휴가제, 반차, 근로자의날 휴무 • 조직 문화 : 회식강요 안함, 야근강요 안함, 자유복장 • 교육/생활 : 워크샵, 교육비 지원, 구내식당(사원식당), 점심식사 제공, 간식 제공, 음료제공(차,커피), 우리사주제도 • 선물 : 명절선물/귀향비","• 입사지원 서류에 허위사실이 발견될 경우, 채용확정 이후라도 채용이 취소될 수 있습니다. • 모집분야별로 마감일이 상이할 수 있으니 유의하시길 바랍니다.",경력 8~10년,대학교졸업(4년) 이상,2025-10-14,"서울 송파구 법원로128, 10층 씨1007호, 씨1008호 지도보기 · 주소복사",https://jumpit.saramin.co.kr/position/51796958,51796958,https://jumpit.saramin.co.kr/company/MjY

In [0]:
df_all_path = f"/dbfs/FileStore/jumpit_posting_{datetime.now(SEOUL).strftime('%Y%m%d')}.csv"
df_all.to_csv(df_all_path, index=False)
print(f"상세 페이지 총합 CSV 저장: {df_all_path}")

상세 페이지 총합 CSV 저장: /dbfs/FileStore/jumpit_posting_20250916.csv


In [0]:
# 크롤링 결과 및 csv 저장 확인
df = pd.read_csv(f"/dbfs/FileStore/jumpit_posting_{datetime.now(SEOUL).strftime('%Y%m%d')}.csv")
print(df.head())

              posting_title  ... job_category
0             웹서버 개발자 신입 채용  ...            1
1         Back-End Engineer  ...            1
2          백엔드 개발자(경력8~10년)  ...            1
3            백엔드 개발자 채용[신입]  ...            1
4  Senior Backend Developer  ...            1

[5 rows x 18 columns]


In [0]:
insert_data(data=df,
            schema="bronze",
            table="brz_jumpit_posting",
            mode="append",
            target="cloud")

### Posting ID별 크롤링 시각 저장 및 DB 적재

In [0]:
df_id = df[['posting_id']].copy()
df_id['posting_id_collection_time'] = datetime.now(SEOUL).strftime("%Y-%m-%d")
df_id.head()

,posting_id,posting_id_collection_time
0,51723619,2025-09-16
1,51807381,2025-09-16
2,51796958,2025-09-16
3,51804885,2025-09-16
4,51835524,2025-09-16


In [0]:
df_path = f"/dbfs/FileStore/jumpit_posting_id_{datetime.now(SEOUL).strftime('%Y%m%d')}.csv"
df_id.to_csv(df_path, index=False)
print(f"상세 페이지 총합 CSV 저장: {df_path}")

상세 페이지 총합 CSV 저장: /dbfs/FileStore/jumpit_posting_id_20250916.csv


In [0]:
insert_data(data=df_id,
            schema="bronze", 
            table="brz_jumpit_posting_id",
            mode="append",
            target="cloud")

## 기업정보 상세페이지 크롤링

In [0]:
urls_all = []

for cid in list(JOB_CATEGORIES.keys()):
    detail_csv = f"/dbfs/FileStore/jumpit_posting_job{cid}_all_{datetime.now(SEOUL).strftime('%Y%m%d')}.csv"
    if not os.path.exists(detail_csv):
        raise FileNotFoundError(f"상세 CSV가 없습니다: {detail_csv}  (먼저 run_detail_for_category({cid}) 실행)")
    
    df_detail = pd.read_csv(detail_csv)
    
    urls = [u for u in df_detail.get("company_url_jumpit", []).astype(str) if u and u.startswith("http")]
    print(f"[{cid}] 기업 상세 URL {len(urls)}건")

    urls_all.extend(urls)

urls_all = sorted(set(urls_all))
print(urls_all)
#.get() 메서드 사용 이유: df['column_name']과 같은 대괄호 문법은 만약 'column_name'이 DataFrame에 없을 경우 KeyError를 발생시켜 프로그램이 중단될 수 있음
#"key": 가져오려는 열의 이름. 여기서는 "company_url_jumpit"
#"default_value": 해당 열이 존재하지 않을 때 반환될 기본값. 여기서는 **빈 리스트 []**가 설정되어 있음.

[1] 기업 상세 URL 64건
[2] 기업 상세 URL 44건
[3] 기업 상세 URL 9건
[4] 기업 상세 URL 8건
[5] 기업 상세 URL 0건
[6] 기업 상세 URL 0건
[7] 기업 상세 URL 6건
[8] 기업 상세 URL 51건
[9] 기업 상세 URL 40건
[10] 기업 상세 URL 14건
[11] 기업 상세 URL 8건
[12] 기업 상세 URL 8건
[13] 기업 상세 URL 30건
[15] 기업 상세 URL 72건
[16] 기업 상세 URL 12건
[17] 기업 상세 URL 6건
[18] 기업 상세 URL 5건
[19] 기업 상세 URL 14건
[20] 기업 상세 URL 1건
[21] 기업 상세 URL 22건
[22] 기업 상세 URL 4건
['https://jumpit.saramin.co.kr/company/MTA0ODE1NzYzNA==?company_nm=쿠도커뮤니케이션', 'https://jumpit.saramin.co.kr/company/MTA0ODExODgyMA==?company_nm=메타넷디지털', 'https://jumpit.saramin.co.kr/company/MTA2ODE4MDg3OA==?company_nm=신보', 'https://jumpit.saramin.co.kr/company/MTA2ODYwOTc5Mg==?company_nm=아이티로그인', 'https://jumpit.saramin.co.kr/company/MTA3ODE4MDEwMA==?company_nm=싸이버로지텍', 'https://jumpit.saramin.co.kr/company/MTA3ODY1MTAwOA==?company_nm=엔에스스마트', 'https://jumpit.saramin.co.kr/company/MTA3ODcyODQxMg==?company_nm=가야데이터', 'https://jumpit.saramin.co.kr/company/MTAxODY2ODM2NA==?company_nm=헬스커넥트', 'https://jumpit.saramin.

### 기업 상세 파서 & 크롤러

In [0]:
# COMMAND ----------
def only_digits(s):
    return "".join(ch for ch in s if ch.isdigit()) 
    #입력받은 문자열 s의 각 문자를 하나씩 ch 변수에 할당 
    #isdigit()는 문자가 '0'부터 '9' 사이의 숫자인 경우에만 True를 반환
    #join() 메서드는 주어진 순회 가능한 객체의 각 항목을 빈 문자열("")을 구분자로 사용하여 하나의 문자열로 결합
    #예: 채용 중인 포지션 123건 -> '1', '2', '3' -> "123"


def parse_company_html(html, url):
    soup = BeautifulSoup(html, "lxml")
    root = soup.select_one('div.sc-86740693-1.jhbtgz') \
        or soup.select_one('[class*="sc-86740693-"][class*="jhbtgz"]') \
        or soup

    # 1) 기업명
    name_el = root.select_one('section h1') or root.select_one('h1')
    기업명 = name_el.get_text(strip=True) if name_el else ""

    # 2) 기업키워드 (상단 태그 리스트)
    키워드 = []
    for li in root.select('ul.kFhGDn li, ul.sc-86740693-3 li'):
        txt = li.get_text(" ", strip=True)
        if not txt: 
            continue
        # 이모지/공백 정리
        txt = re.sub(r'^[^\w가-힣]+', '', txt).strip()
        if txt:
            키워드.append(txt)
    # 중복 제거 & JSON 문자열로 저장 (요청 포맷 예시에 맞춤)
    기업키워드 = json.dumps(list(dict.fromkeys(키워드)), ensure_ascii=False)
    #dict.fromkeys(키워드) = 키워드 리스트의 모든 요소를 딕셔너리의 키로 변환 (순서유지)     #키 값만 다시 리스트로 변환 #json.dumps()는 파이썬 객체를 JSON 형식의 문자열로 변환

    # 3) 채용 중인 포지션 수
    pos_text = ""
    more = root.select_one('#COMPANY_INFO .more')
    if more:
        pos_text = more.get_text(" ", strip=True)
    if not pos_text:
        num_el = root.select_one('#COMPANY_INGPOSITION em.num')
        pos_text = num_el.get_text(strip=True) if num_el else ""
    채용중인포지션수 = None
    m = re.search(r'(\d+)\s*건', pos_text)
    #re.search()나 re.match()와 같은 정규 표현식 함수는 패턴과 일치하는 부분을 찾으면 match object라는 객체를 반환
    if m:
        채용중인포지션수 = int(m.group(1))
        # m: 매치 객체(match object)
        #.group(1): 매치 객체의 메서드
        #.group(0) 또는 .group(): 패턴 전체와 일치하는 문자열 반환
        #.group(1): 첫 번째 캡처 그룹(첫 번째 괄호 ())과 일치하는 문자열 반환
        #.group(2): 두 번째 캡처 그룹과 일치하는 문자열 반환

    else:
        # 숫자만 있을 경우
        d = only_digits(pos_text)
        채용중인포지션수 = int(d) if d.isdigit() else None

    # 4) 업력 / 설립연도 / 주소 (COMPANY_INFO dl)
    업력 = ""
    설립연도 = ""
    주소 = ""
    company_info = root.select_one('#COMPANY_INFO dl, .ipTgkz dl')
    for div in company_info.select('div'):
        dt = div.select_one('dt')
        dd = div.select_one('dd')
        if not dt or not dd: 
            continue
        key = dt.get_text(strip=True)
        val_text = dd.get_text(" ", strip=True)
        if key == "업력":
            업력 = clean_text(val_text)
            ym = re.search(r'(\d{4})\s*년', val_text)
            if ym:
                설립연도 = ym.group(1)
        elif key in ["대표주소","주소"]:
            strong = dd.select_one('strong')
            주소 = clean_text(strong.get_text(" ", strip=True) if strong else val_text)
            #.get_text(...): 지정된 HTML 태그와 그 안에 포함된 모든 자식 태그의 텍스트 콘텐츠를 가져오는 메서
            #" " (구분자): get_text()의 여러 자식 태그의 텍스트를 하나로 합칠 때 이 구분자를 사용
            #strip=True: strip 매개변수는 텍스트의 앞뒤에 있는 불필요한 공백이나 줄바꿈을 제거하는 옵션

    # 5) 홈페이지 URL
    홈페이지URL = ""
    # (1) dt가 '홈페이지'인 경우 우선
    for dl in root.select('#COMPANY_INFO dl'):
        dt = dl.select_one('dt')
        dd = dl.select_one('dd')
        if dt and dd and "홈페이지" in dt.get_text(strip=True):
            a = dd.select_one('a[href^="http"]')
            #.select_one(...): Beautiful Soup의 메서드로, 주어진 CSS 선택자와 일치하는 첫 번째 요소를 찾아 반환
            #'a[href^="http"]': 이것이 바로 CSS 속성 선택자
            #a: <a> 태그(앵커, 링크)를 선택합니다.
            #[href^="..."]: href 속성이 특정 문자열로 시작하는 요소를 선택하는 문법
            #"http": href 속성이 "http" (또는 "https")로 시작하는 것을 의미
            
            if a:
                홈페이지URL = a["href"].strip()
                break
            #a: 이 변수는 앞선 select_one() 코드를 통해 찾은 <a ...> HTML 태그를 나타내는 Beautiful Soup 객체
            # a["href"]: 파이썬 딕셔너리와 유사한 문법으로, a 객체에서 href라는 속성의 값을 가져옴. 예를 들어, <a href="https://example.com" ...> 태그에서 a["href"]는 "https://example.com"이라는 문자열을 반환

    # (2) 대체: 점핏 외부 도메인 첫번째 링크 사용
    if not 홈페이지URL:
        for a in root.select('a[href^="http"]'):
            href = a.get("href","")
            if href and "jumpit" not in href:
                홈페이지URL = href #홈페이지 주소 없으면 지도 주소라도 들어갈 듯
                break #하나만 찾으면 넘김

    # 6) 기술스택 (COMPANY_TECHSTACK)
    techs = [clean_text(x.get_text(" ", strip=True)) 
             for x in root.select('#COMPANY_TECHSTACK .stack_wrap [class*="sc-d9de2de1-0"], #COMPANY_TECHSTACK .stack_wrap div')]
    890#이 리스트에는 빈 문자열이나 중복된 값이 포함될 수 있음

    techs = [t for t in techs if t] #리스트에서 비어 있는 값("", None 등) 제거
    기술스택 = ", ".join(dict.fromkeys(techs)) #딕셔너리로 만들어 중복 제거. dict.fromkeys()가 반환한 딕셔너리({'Python': None, 'Java': None})의 키들을 가져와 쉼표와 공백(",")으로 연결하여 하나의 문자열로 만듦

    # 7) 복지 및 혜택
    복지및혜택 = ""
    # 회사 페이지에는 별도 섹션명이 다를 수 있어 헤더 탐색
    welfare_section = None
    for sec in root.select('section'):
        h2 = sec.select_one('h2')
        if h2 and ("복지" in h2.get_text(strip=True) or "혜택" in h2.get_text(strip=True)):
            welfare_section = sec
            break
    if welfare_section:
        # 리스트/문단/태그 텍스트 취합
        txts = []
        for node in welfare_section.select('li, dd, p, pre, div'):
            t = node.get_text("\n", strip=True)
            if t:
                txts.append(t)
        if txts:
            복지및혜택 = clean_text("\n".join(txts)) #txts 리스트의 모든 항목을 줄바꿈 문자(\n)로 연결하여 하나의 큰 문자열로 만듦. 

    collected_at = datetime.now(SEOUL).strftime("%Y-%m-%d %H:%M:%S")

    return {
        "company_name": 기업명,
        "company_keywords_jumpit": 기업키워드,   # JSON 문자열 형태
        "active_positions": 채용중인포지션수,
        "years_in_business": 업력,
        "founded_year": 설립연도,
        "address": 주소,
        "homepage_url": 홈페이지URL,
        "company_tech_stack": 기술스택,
        "company_benefits": 복지및혜택,
        "company_url_jumpit": url,
        "jumpit_company_collection_time": collected_at
    }

def crawl_company_pages(urls, delay_range=DETAIL_DELAY):
    rows = []
    driver = start_driver() #Selenium 드라이버 인스턴스 시작
    for i, url in enumerate(urls, 1):
        try:
            driver.get(url) #현재 URL로 이동하여 페이지를 로드
            WebDriverWait(driver, 25).until(
                EC.any_of(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.sc-86740693-1.jhbtgz')),
                    EC.presence_of_element_located((By.CSS_SELECTOR, '[class*="sc-86740693-"][class*="jhbtgz"]'))
                )
            ) #페이지의 핵심 요소가 로드될 때까지 최대 25초 대기
            rows.append(parse_company_html(driver.page_source, url))
            #현재 페이지의 HTML 소스를 가져와 parse_company_html 함수로 전달
        except TimeoutException:
            rows.append({
                "company_name":"", "company_keywords_jumpit":"[]", "active_positions":None, "years_in_business":"", "founded_year":"",
                "address":"", "homepage_url":"", "company_tech_stack":"", "company_benefits":"", "company_url_jumpit":url, "jumpit_company_collection_time":""
            })
        finally:
            time.sleep(random.uniform(*delay_range))
            #*delay_range: 언패킹 연산자(unpacking operator). delay_range 변수는 (0.7, 1.5)와 같은 튜플 형태로 되어 있는데, *를 붙이면 튜플의 요소들이 개별 인자로 풀려서 random.uniform() 함수에 전달됨
    driver.quit()
    return rows

### 기업 상세 전량 크롤링 -> CSV 저장 및 DB 적재

In [0]:
# COMMAND ----------
# 기업 상세 크롤링
company_rows = crawl_company_pages(urls_all)
cdf = pd.DataFrame(company_rows)

# 스키마/컬럼 순서 정렬
company_cols = [
    "company_name","company_keywords_jumpit","active_positions","years_in_business","founded_year","address","homepage_url","company_tech_stack","company_benefits","company_url_jumpit", "jumpit_company_collection_time"
]
cdf = cdf.reindex(columns=company_cols) #열 인덱스 정렬

display(cdf)
print("총 기업 수:", len(cdf))

# CSV 저장
company_csv = "/dbfs/FileStore/jumpit_company_{datetime.now(SEOUL).strftime('%Y%m%d')}.csv"
cdf.to_csv(company_csv, index=False)
print("기업 상세 페이지 CSV 저장:", company_csv)

company_name,company_keywords_jumpit,active_positions,years_in_business,founded_year,address,homepage_url,company_tech_stack,company_benefits,company_url_jumpit,jumpit_company_collection_time
쿠도커뮤니케이션,"[""매출액 500억 기업""]",16.0,26 년차 ( 2000년 10월 설립 ),2000,"경기 과천시 과천대로7나길 9, 12층 (갈현동,디엑스타워)","https://m.map.naver.com/search2/search.nhn?query=경기 과천시 과천대로7나길 9, 12층 (갈현동,디엑스타워)#/map","CSS 3, Java, AngularJS, Oracle PL/SQL, PostgreSQL, Headless UI, React, SQL, Jenkins, REST API, Apache Sling, XML, JavaScript, HTML5, MongoDB, Oracle, Adobe Target","단축 근무제도 열심히 일하는 당신! 30분 늦게 출근하세요♥ Family Day 한 달에 한 번! 금요일은 조기 퇴근 남여 휴게실 제공 (최신형 안마의자) 쉬고 싶을 때는 편하게! 프라이빗 공간에서 안마의자 이용 생일 조기 퇴근 및 생일 상품권 제공 생일 축하합니다! 소정의 상품권도 드려요 부모 부양지원금 경조사 지원 •자녀 입학축하금 쿠도 영웅 포상 장기근속포상 종합건강검진 지원 단체상해보험 가입 커피,음료 무제한 제공 샌드위치 데이 라운지 스낵바 (출출박스) 사내 동호회 활동 지원 모범사원포상 웰컴키트 지원 CUDO 행복배달 자격증 취득 축하금 00000 법인 리조트 운영 서울랜드 입장료 지원 외국어 교육/도서 구입비 지원 기념일 선물 (명절,성탄절, 정월대보름)",https://jumpit.saramin.co.kr/company/MTA0ODE1NzYzNA==?company_nm=쿠도커뮤니케이션,2025-09-16 10:52:07
메타넷디지털,"[""연봉상승률 10% 이상"", ""1호선 역세권 기업"", ""평균연봉 6,000 이상""]",5.0,37 년차 ( 1989년 4월 설립 ),1989,서울 종로구 종로33길 15 2F,https://m.map.naver.com/search2/search.nhn?query=서울 종로구 종로33길 15 2F#/map,"AI/인공지능, Azure Bot Service, ProProfs Knowledge Base, R Language, Azure Cognitive Search, Python","○ Metanet 기업문화 프로그램 1. 몸과 마음의 건강을 케어 해주는 프로그램 : 건강검진, 심리/상담 프로그램 등 2. 시즌에 맞게 즐거운 여가를 보낼 수 있는 가족, 개인별 선택형 프로그램 : Metanet Family Picnic, 역사탐방, 키자니아 체험 등 3. 충분한 쉼과 재충전을 보장해주는 프로그램 : 법인 콘도 이용, 힐링 Award 등 4. 중요한 삶의 순간을 축하와 위로, 다양한 지원으로 함께해주는 프로그램 : 단체상해보험, 경조사, 장례지원서비스, 명절 선물 등 5. 구성원들의 역량강화를 위한 지속적인 자기개발 지원 프로그램 : 사내 전자 도서관, 온/오프라인 교육 지원, 자격증 취득 지원, 자격수당 지급 등",https://jumpit.saramin.co.kr/company/MTA0ODExODgyMA==?company_nm=메타넷디지털,2025-09-16 10:52:09
신보,"[""매출액 5000억 기업"", ""5호선 역세권 기업""]",1.0,27 년차 ( 1999년 7월 설립 ),1999,서울 마포구 백범로 235(신보빌딩),https://m.map.naver.com/search2/search.nhn?query=서울 마포구 백범로 235(신보빌딩)#/map,"AZURE, Linux, CISSP, Windows, CISA, AWS","• 종합 건강 진단비 지원 - 임직원 본인 및 직계가족의 질병을 조기 발견하여 치료할 수 있는 기회를 제공 • 휴양시설 - 휴양시설을 활용할 수 있는 기회를 제공하여 구성원들이 편안한 여가생활을 즐길 수 있도록 지원 • 학자금 - 임직원의 취학 전 자녀의 보육비 지원 • 경조사 지원 - 임직원 또는 가족의 경조사 발생에 따른 축하 또는 위로의 의미를 전달하기 위하여 축하금/위로금/상조용품 및 휴가 부여 • 동호회 지원 - 임직원들의 친목을 도모하고 건전한 자기계발의 기회 제공 • 기념품 지급 - 생일, 창립기념일, 근로자의 날, 설날, 추석 등 임직원에게 기념품을 지급 • 생일파티 - 매월 임직원의 생일을 축하하기 위한 간단한 생일파티를 진행하며, 생일자에게는 상품권 증정 • 조직문화 활동비 지원 - 멘토링(선후배 직급간) 및 부서/동료간 활동비 지원 • 교육비 지원 - 임직원의 직무역량 향상과 경력개발을 위한각종 교육비 지원 • 해외현장 근무자 최고 대우 - 동종업계 최고수준(대기업 수준)의 급여, 파견근무 수당(체재비 별도 지급) • 연봉 수준 : 회사내규에 따름 (신입초봉 4,364만원)",https://jumpit.saramin.co.kr/company/MTA2ODE4MDg3OA==?company_nm=신보,2025-09-16 10:52:11
아이티로그인,"[""육아휴직"", ""장기근속 포상"", ""자유로운 연차"", ""6호선 역세권 기업"", ""간식 제공"", ""성과급/상여금"", ""사내 카페""]",1.0,24 년차 ( 2002년 4월 설립 ),2002,"서울 용산구 원효로 138, 403호","https://m.map.naver.com/search2/search.nhn?query=서울 용산구 원효로 138, 403호#/map","Network, Cisco, Infra, HP Cloud Compute, Ccnp","ㆍ지원금/보험 : 건강검진, 직원대출제도, 각종 경조사 지원, 복지카드/포인트 ㆍ급여제도 : 퇴직연금, 인센티브제, 상여금, 장기근속자 포상, 우수사원포상, 퇴직금, 4대 보험 ㆍ선물 : 명절선물/귀향비, 창립일선물지급, 생일선물/파티, 생일자 조기퇴근 ㆍ교육/생활 : 창립일행사, 우수사원시상식, 워크샵, 간식 제공, 음료제공(차, 커피) ㆍ근무 환경 : 휴게실, 회의실, 카페테리아, 사원증, 자회사 제품할인, 사무용품 지급 ㆍ조직문화 : 자유로운 연차사용 ㆍ출퇴근 : 차량유류비지급, 주차비지원, 회사차량 있음 ㆍ리프레시 : 연차, 경조휴가제, 반차, 창립일휴무, 근로자의 날 휴무, 포상휴가, 산전 후 휴가, 육아휴직",https://jumpit.saramin.co.kr/company/MTA2ODYwOTc5Mg==?company_nm=아이티로그인,2025-09-16 10:52:14
싸이버로지텍,"[""평균연봉 상위 10%"", ""매출액 500억 기업"", ""주거복지"", ""장기근속 포상"", ""유수 계열사"", ""재택근무"", ""경의선 역세권 기업"", ""유연근무제"", ""점심지원"", ""평균연봉 7,000 이상""]",6.0,26 년차 ( 2000년 3월 설립 ),2000,"서울 마포구 월드컵북로 396, 16층 (상암동,누리꿈스퀘어연구개발타워)","https://m.map.naver.com/search2/search.nhn?query=서울 마포구 월드컵북로 396, 16층 (상암동,누리꿈스퀘어연구개발타워)#/map","Java, AI/인공지능, RPA, Linux, XML, JavaScript, HTML5, Spring Boot, FastAPI, Windows, Oracle","• 주1회 재택근무 시행 중 • 최신형 LG Gram 노트북 지급 • 유연근무제도 및 자율퇴근 • 복지카드 충전(직급 따라 연간 120~180만원 적립) • 중식비 지원(복지카드에 매월 근무일수 * 10000원 만큼 매달 1일 적립) • 저축지원금(매월 고정급의 2% 추가 적립) • 생활안정보험금(월 불입액에 50%, 최대 5만원) • 건강검진 지원(연1회) • 장기근속에 따른 안식휴가 제도 운영(5,10,15년차)",https://jumpit.sar

총 기업 수: 120
기업 상세 페이지 CSV 저장: /dbfs/FileStore/jumpit_company_{datetime.now(SEOUL).strftime('%Y%m%d')}.csv


In [0]:
# 크롤링 결과 및 csv 저장 확인
df = pd.read_csv("/dbfs/FileStore/jumpit_company_{datetime.now(SEOUL).strftime('%Y%m%d')}.csv")
print(df.head())

  company_name  ... jumpit_company_collection_time
0     쿠도커뮤니케이션  ...            2025-09-16 10:52:07
1       메타넷디지털  ...            2025-09-16 10:52:09
2           신보  ...            2025-09-16 10:52:11
3       아이티로그인  ...            2025-09-16 10:52:14
4       싸이버로지텍  ...            2025-09-16 10:52:16

[5 rows x 11 columns]


In [0]:
insert_data(data=df,
            schema="bronze", 
            table="brz_jumpit_company",
            mode="append",
            target="cloud")